Tutorial : Building a Multi-Agent Chat System with Azure OpenAI, Autogen and Tracing using Phoenix

Introduction : In this tutorial, we'll guide you through building a multi-agent chat system using Azure OpenAI deployments. The example in this tutorial is a customer query resolver agent. The system will utilize multiple AI Agents, each with a specific role, to colaboratively come up with a solution to user query. You'll see how to co-ordinate these agents in a RoundRobin Fashion, trace their interactions with Phoenic and add human annotations for debugging and optimization. The applications of multi-agent systems are very diverse. The Agents are capable of handing both TextMessage or MultiModalMessage. Autogen AgentChat provides a set of preset Agents, each with variations on how an agent might respond to messages.

Refer <a href = https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html>Autogen</a> Documentation for more details about AgentChat 

Prequisites : Python, Access to OpenAI models. Other dependencies are listed in requirements.txt. 
Run pip install -r /path/to/requirements.txt to have all packages installed at once


In [1]:
#It is a good practice to save all api keys, endpoints and other important details in an env file. 
from dotenv import load_dotenv
_ = load_dotenv("env")

Launch the Phoenix App

In [2]:
# Launch Phoenix
import os
if "PHOENIX_API_KEY" in os.environ:
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

else:
    import phoenix as px

    px.launch_app().view()

# Connect to Phoenix
from phoenix.otel import register
tracer_provider = register()


/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Instrument OpenAI for tracing

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

In many cases, agents need access to LLM model services such as OpenAI, Azure OpenAI, or local models. Since there are many different providers with different APIs, autogen-core implements a protocol for model clients and autogen-ext implements a set of model clients for popular model services. AgentChat can use these model clients to interact with model services.

To use the Azure OpenAI, you need to provide your deployment id, Azure Cognitive Services endpoint, api version, and model capabilities. For authentication, you can either provide an API key or an Azure Active Directory (AAD) token credential.

Step 1 : Set up your Azure OpenAI Client

In [ ]:
import os
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

 

#Set up Azure OpenAI Configuration using environment variables

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

AZURE_OPENAI_API_VERSION = os.getenv("API_VERSION")

 
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="model_name",
    model="model_name",
    api_version="your_model_version",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0.2,
    max_tokens=200
)


Step 2 : Define Agent Functions
Each Agent is initialized using the AssistantAgent class from Autogen. It's up to the developer to decide the system message according to their needs
Here I have used three agents - Classifier, Resolver and Feedback


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination # type: ignore
from autogen_agentchat.teams import RoundRobinGroupChat 

classifier_agent = AssistantAgent(

    name="classifier",
    model_client =az_model_client,

    system_message=("You are a classifier that determines the category og a customer query"
                    "The categories are Billing, Technical Support, Shipping or General Inquiry"
                    "Respond only with categoy name")
)


problem_solver_agent = AssistantAgent(

    name="problem_solver",

    model_client = az_model_client,

    system_message=("You are a resolver that answers customer queries based on their category."
                    "You will receive the category and query, and you must provide a resolution"
                    "Be concise, clear, and empatheic in your response"
                    "Try to limit the answers in 5 lines")

)

feedbackagent = AssistantAgent(
    name = "feedback_agent",
     model_client=az_model_client,
    system_message=(
        "You analyse customer interactions for sentiments and suggest improvements"
        "If the sentiment is negative, identify and propose better response"
        "If all looks good respond with TERMINATE"
    )
)

 

Step 3 : Now we set up a team using the RoundRobinGroupChat configuration. This configuration allows multiple agents to work together in a round-robin fashion, where each agent takes turns responding while maintaining a shared context.

The AssistantAgent is responsible for generating responses based on the given input. Additionally, we will set a TextMentionTermination condition that stops the team when a specific word is detected in any of the agent's responses.


In [5]:
termination =TextMentionTermination("TERMINATE")
group_chat1 = RoundRobinGroupChat(
    [classifier_agent, problem_solver_agent, feedbackagent], termination_condition=termination)

In [7]:
from phoenix.trace import TraceDataset
from autogen_agentchat.ui import Console


In [10]:
query = "who do i contact for broken screen?"
await Console(group_chat1.run_stream(task=query))


/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='who do i contact for broken screen?', type='TextMessage'), TextMessage(source='classifier', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=2), content='Technical Support', type='TextMessage'), TextMessage(source='problem_solver', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=55), content="I'm sorry to hear about your broken screen. To get it fixed, please contact the technical support team of the device's manufacturer. If you have warranty or insurance, make sure to mention that when you reach out. They will guide you through the repair or replacement process.", type='TextMessage'), TextMessage(source='feedback_agent', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=3), content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

In [11]:

group_chat2 = RoundRobinGroupChat(
    [classifier_agent, problem_solver_agent, feedbackagent], termination_condition=termination)

In [12]:
query1 = "My payment isnt going through on your app"
await Console(group_chat2.run_stream(task=f"Query: {query1}"))

/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Query: My payment isnt going through on your app', type='TextMessage'), TextMessage(source='classifier', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=1), content='Billing', type='TextMessage'), TextMessage(source='problem_solver', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=62), content="I apologize for the inconvenience you're experiencing with the payment. Please ensure your card details are entered correctly and that your card is not expired. If the issue persists, try using a different payment method or contact our billing support for further assistance. They will be able to help resolve any payment issues you're facing.", type='TextMessage'), TextMessage(source='feedback_agent', models_usage=RequestUsage(prompt_tokens=219, completion_tokens=3), content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

In [13]:
query2 = "I cant log into my account"
await Console(group_chat2.run_stream(task=f"Query: {query2}"))

/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(
/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4. Model mapping may be incorrect.
  result = await self._model_client.create(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Query: I cant log into my account', type='TextMessage'), TextMessage(source='classifier', models_usage=RequestUsage(prompt_tokens=173, completion_tokens=2), content='Technical Support', type='TextMessage'), TextMessage(source='problem_solver', models_usage=RequestUsage(prompt_tokens=260, completion_tokens=71), content="I'm sorry to hear you're having trouble logging in. Please check if you're using the correct username and password. If you've forgotten your password, use the 'Forgot Password' feature to reset it. If the problem persists, please reach out to our technical support team for assistance. They'll be happy to help you regain access to your account.", type='TextMessage'), TextMessage(source='feedback_agent', models_usage=RequestUsage(prompt_tokens=328, completion_tokens=3), content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

Step 4 : Open the phoenix url that opened in your localhost, you will now see the traces for all the queries that we ran.


<img src = "img7.png">

In [16]:
my_traces = px.Client().get_trace_dataset().save()

💾 Trace dataset saved to under ID: 8aa487d2-7cfb-4dc0-a762-9d134f6a5dd8
📂 Trace dataset path: /home/codespace/.phoenix/trace_datasets/trace_dataset-8aa487d2-7cfb-4dc0-a762-9d134f6a5dd8.parquet


In [15]:
my_traces

UUID('a4290b62-2880-45b8-af9b-d5db4fde4323')

In [17]:
px.launch_app(trace=px.TraceDataset.load(my_traces))

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [19]:
pip freeze > requirements.txt 

I0000 00:00:1736784018.404160    1550 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.
